'''

We appreciated your help in stepping in during a bit of an exigent situation. We have a bit calmer of a task for you and one suited to a "Sun Devil." We have some basic crime data for Phoenix and we need to make better sense of it. We want to know where different kinds of crimes are occurring, in which areas crime is growing fastest (or dropping fastest), and whether certain crimes are more common in certain areas of the city. Basically, we don't need maps or anything at this stage, just some data grouped by location (either the type of location or the zip codes) and some trend data.

I mean, if you have the time for a bit of a challenge, we would love for you to bring in some other data to help draw a better picture around this. Are there some factors that affect the crime rate? If there are, we could see if there were ways to see where crime was more likely. We might even ask you to head up our new Pre-Crime unit in the Valley.

'''

*For this task, we want to:*

Grouping crimes by location type or by zip code (or groups of zip codes).

Looking at types of crimes and where they are most common.

Find our which areas have the fastest growing and shrinking crime rates.

Grouping crimes by violent and non-violent crimes.

In [2]:
#import dependencies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [18]:
#load data
file_path = r'C:\Users\denee\Desktop\Homework\crime-data_crime-data_crimestat.csv'

# Specify the data types  
df = pd.read_csv(file_path)
df.head()

,INC NUMBER,OCCURRED ON,OCCURRED TO,UCR CRIME CATEGORY,100 BLOCK ADDR,ZIP,PREMISE TYPE,GRID
0,198181008285,1/20/2023 0:00,NaN,BURGLARY,60XX S 11TH ST,85042.0,SINGLE FAMILY HOUSE,AC29
1,198282025394,6/25/2023 0:00,6/25/2023 0:00,LARCENY-THEFT,16XX N BLACK CANYON AC,85009.0,PARKING LOT,BD23B
2,198282029471,6/24/2023 0:00,6/24/2023 0:00,BURGLARY,5XX W CAMELBACK RD,85013.0,SPECIALTY STORE,BI27
3,198282030338,6/25/2023 0:00,NaN,LARCENY-THEFT,4XX N 17TH AVE,85007.0,SINGLE FAMILY HOUSE,BB25
4,198282030737,6/25/2023 0:00,6/25/2023 0:00,LARCENY-THEFT,48XX E BRILL ST,85008.0,SINGLE FAMILY HOUSE,BC39


Grouping crimes by location type or by zip code (or groups of zip codes).

In [26]:
#Grouping crimes by zip code (or groups of zip codes).
crime_count_by_zip = df.groupby(['ZIP', 'PREMISE TYPE']).size()

# Print the result
print(crime_count_by_zip)

ZIP      PREMISE TYPE                       
85003.0  ABANDONED/CONDEMNED STRUCTURE             3
         AIRPORT                                   3
         ALLEY                                    39
         APARTMENT                              1094
         ARENA / STADIUM / CONVENTION CENTER      10
                                                ... 
85396.0  SINGLE FAMILY HOUSE                       1
         SPECIALTY STORE                           2
         STREET / ROADWAY / ALLEY / SIDEWALK       3
         STREET / ROADWAY / SIDEWALK               1
         VEHICLE                                   2
Length: 4084, dtype: int64


Looking at types of crimes and where they are most common.

In [27]:
# Group by 'UCR CRIME CATEGORY' and then 'PREMISE TYPE'
grouped = df.groupby(['UCR CRIME CATEGORY', 'PREMISE TYPE'])

# Count the occurrences of crime in each group
crime_counts = grouped.size()

# Convert the series to a df
crime_counts_df = crime_counts.reset_index(name='count')

# Sort the df by CRIME CATEGORY in descending order
sorted_crime_counts = crime_counts_df.sort_values(['UCR CRIME CATEGORY', 'count'], ascending=[True, False])

# Drop duplicates, keeping the most frequent for each CRIME CATEGORY'
most_frequent_premise = sorted_crime_counts.drop_duplicates(subset=['UCR CRIME CATEGORY'])

print(most_frequent_premise)

                        UCR CRIME CATEGORY         PREMISE TYPE  count
5                       AGGRAVATED ASSAULT            APARTMENT  11888
148                                  ARSON  SINGLE FAMILY HOUSE    503
225                               BURGLARY  SINGLE FAMILY HOUSE  28352
321                           DRUG OFFENSE              VEHICLE   7709
328                          LARCENY-THEFT            APARTMENT  25767
426                    MOTOR VEHICLE THEFT            APARTMENT  13748
543  MURDER AND NON-NEGLIGENT MANSLAUGHTER  SINGLE FAMILY HOUSE    241
617                                   RAPE  SINGLE FAMILY HOUSE   2349
684                                ROBBERY          PARKING LOT   3131


Find our which areas have the fastest growing and shrinking crime rates.

In [28]:
#Convert "OCCURRED ON" to a date time column in pandas
df['OCCURRED ON'] = pd.to_datetime(df['OCCURRED ON'])

# Extract the year (or you can choose month, quarter, etc.)
df['Year'] = df['OCCURRED ON'].dt.year

In [29]:
# Group by ZIP and Year, then count the number of crimes
crime_counts = df.groupby(['ZIP', 'Year']).size().reset_index(name='Count')

In [30]:
# Pivot so that Years is a column and ZIP is rows
pivot_crime_counts = crime_counts.pivot(index='ZIP', columns='Year', values='Count')

In [31]:
# Calculate the year-over-year difference in crime counts by ZIP
pivot_crime_counts = pivot_crime_counts.diff(axis=1)

# Find the ZIP with the greatest increase in crime for each year
max_increase_zip = pivot_crime_counts.idxmax()

print(max_increase_zip)

Year
2015.0        NaN
2016.0    85015.0
2017.0    85043.0
2018.0    85007.0
2019.0    85041.0
2020.0    85008.0
2021.0    85029.0
2022.0    85004.0
2023.0    85021.0
2024.0    85260.0
dtype: float64


Here we see that the zips 85021 and 85260 have been having the greatest increase in crime in the recent years (2023-2024)

Now, lets group crimes by violent and non-violent crimes.

In [33]:
print(df['UCR CRIME CATEGORY'].unique())

['BURGLARY' 'LARCENY-THEFT' 'MOTOR VEHICLE THEFT' 'ROBBERY' 'ARSON'
 'AGGRAVATED ASSAULT' 'RAPE' 'DRUG OFFENSE'
 'MURDER AND NON-NEGLIGENT MANSLAUGHTER']


We see that the values aggrevated assault, rage, murder, and arson can be classified as 'voilent crime', lets have the other values classified as 'non-voilent crimes' in a new binary column in our dataset

In [34]:
violent_crimes = ['AGGRAVATED ASSAULT', 'RAPE', 'MURDER AND NON-NEGLIGENT MANSLAUGHTER', 'ARSON']
df['VIOLENT_CRIME'] = np.where(df['UCR CRIME CATEGORY'].isin(violent_crimes), 1, 0)

In [35]:
# Group by ZIP, Year, and Violent Crime status, and then count the number of crimes
crime_counts = df.groupby(['ZIP', 'Year', 'VIOLENT_CRIME']).size().reset_index(name='Count')

In [36]:
# Pivot the table with ZIP and Violent Crime as rows and Year as columns
pivot_crime_counts = crime_counts.pivot_table(index=['ZIP', 'VIOLENT_CRIME'], columns='Year', values='Count')

In [37]:
# Calculate the year-over-year change in crime count
pivot_crime_counts = pivot_crime_counts.diff(axis=1)

In [38]:
# Reset index 
pivot_crime_counts_reset = pivot_crime_counts.reset_index()

In [39]:
# Filter for violent and non-violent crimes separately and find the ZIP code with the greatest increase
max_increase_violent = pivot_crime_counts_reset[pivot_crime_counts_reset['VIOLENT_CRIME'] == 1].set_index('ZIP').idxmax(axis=1)
max_increase_non_violent = pivot_crime_counts_reset[pivot_crime_counts_reset['VIOLENT_CRIME'] == 0].set_index('ZIP').idxmax(axis=1)

In [40]:
print("ZIP codes with the greatest increase in violent crimes:")
print(max_increase_violent)
print("\nZIP codes with the greatest increase in non-violent crimes:")
print(max_increase_non_violent)

ZIP codes with the greatest increase in violent crimes:
ZIP
85003.0           2016.0
85004.0           2016.0
85006.0           2016.0
85007.0           2016.0
85008.0           2016.0
               ...      
85388.0    VIOLENT_CRIME
85390.0    VIOLENT_CRIME
85392.0    VIOLENT_CRIME
85395.0    VIOLENT_CRIME
85396.0    VIOLENT_CRIME
Length: 98, dtype: object

ZIP codes with the greatest increase in non-violent crimes:
ZIP
85003.0           2016.0
85004.0           2016.0
85006.0           2016.0
85007.0           2016.0
85008.0           2016.0
               ...      
85383.0           2023.0
85388.0    VIOLENT_CRIME
85392.0           2019.0
85395.0           2022.0
85396.0           2018.0
Length: 111, dtype: object
